# Prompt Processing
Using the OpenAI Completion API, I will classify the prompt into categories and format them in a way where I can batch upload to my PromptsLib DB.  
- References: [OpenAI Cookbook](https://cookbook.openai.com/examples/batch_processing)

In [ ]:
import os
import json
from openai import OpenAI
import pandas as pd
# from IPython.display import display, Image
from dotenv import load_dotenv
from pprint import pprint
import requests

In [ ]:
load_dotenv()

In [ ]:
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    organization=os.getenv("OPENAI_ORG_ID"),
    project=os.getenv("OPENAI_PROJECT_ID")
)

In [ ]:
ALL_CATEGORIES = ['Storytelling', 'Poetry', 'Scriptwriting', 'Dialogue Generation', 'Character Development', 'Marketing', 'Sales', 'Customer Support', 'Project Management', 'Business Strategy', 'Language Learning', 'Science and Technology', 'History and Geography', 'Mathematics', 'Exam Preparation', 'Movie Reviews', 'TV Show Summaries', 'Music Recommendations', 'Book Summaries', 'Game Walkthroughs', 'Code Generation', 'Debugging Assistance', 'Software Documentation', 'Tech Tutorials', 'API Usage', 'Fitness Advice', 'Nutrition Tips', 'Mental Health Support', 'Medical Information', 'Lifestyle Coaching', 'Goal Setting', 'Productivity Hacks', 'Time Management', 'Career Advice', 'Motivational Quotes', 'Destination Guides', 'Travel Tips', 'Itinerary Planning', 'Cultural Insights', 'Adventure Activities', 'Personal Finance', 'Investment Strategies', 'Market Analysis', 'Budgeting Tips', 'Cryptocurrency', 'Cooking Recipes', 'Home Improvement', 'Gardening Tips', 'DIY Projects', 'Arts and Crafts', 'Breaking News', 'Political Analysis', 'Sports Updates', 'Economic Trends', 'Environmental News', 'Latest Research', 'Scientific Theories', 'Technological Innovations', 'Space Exploration', 'Environmental Science', 'Fun Facts', 'Trivia', 'Jokes and Humor', 'Inspirational Stories', 'Thought Experiments', 'Misc/Other']

In [ ]:
prompt_processing_system_prompt = f"""
Your goal is assist me in the following tasks, given a prompt: 
- classifying the prompt into categories.
- creating a title for the prompt. We're aiming for conciseness in titles, since this will be used in a list view.
- formatting the prompt in a way where I can batch upload to my PromptsLib DB. 

These are prompts for interacting with Large Language Models (LLMs) like OpenAI GPT-4. The categories are as follows:

{", ".join(ALL_CATEGORIES)}

The output I want is a JSON object with the following structure. Note that the values are all placeholders or examples, except the user_id which should always be set to 1:
{{
    "title": "Title of the Prompt",
    "categories": [{{"name": "Fun Facts"}}, {{"name": "Trivia"}}...],
    "content": "Repeat the full prompt here.",
    "user_id": 1,
    "blob_metadata": {{}}
}}
"""

In [ ]:
pprint(prompt_processing_system_prompt)

In [ ]:
def process_prompt(prompt: str, system_prompt: str = prompt_processing_system_prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        temperature=0.2,
        # Enable JSON mode
        response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [ ]:
trial_prompt = """Act as an expert AWS architect to construct an intricate, comprehensive plan for creating and operating a secure, scalable, and highly efficient cloud-based infrastructure on AWS for a new, dynamic-function software application. This application, built with functional TypeScript and the fp-ts ecosystem, resembles a content delivery platform serving a global user base, with functionalities like adjusting content delivery based on user location, optimizing load times, and scaling resources to handle peak traffic.

The plan should incorporate principles of high availability, fault tolerance, redundancy across various availability zones, and auto-scaling. Considering the application's functionalities, recommend the most suitable data storage solution. Propose advanced strategies for system monitoring and optimization, ensuring adaptability to fluctuating performance needs with a focus on high-performance metrics. In the absence of specific compliance or disaster recovery needs, prioritize creating an expert-level AWS architectural solution with a flexible approach to budget considerations.

In devising strategies, contemplate multiple approaches, factoring in elements like speed, elegance, and performance. Discuss the advantages and potential drawbacks of each approach, making a final decision based on an objective evaluation.

Upon deciding the strategy, develop the solution while conforming to an Agile methodology, demonstrating flexibility and adaptability according to the task's requirements. Highlight functional programming, but remain open to other paradigms if necessary. Your solutions should be clean, efficient, comprehensive, and maintainable. Conclude with insights on potential scalability and performance enhancements of the solution implemented.

Given the global nature of the user base, certain security considerations, like data encryption, network isolation, and regulatory compliance, become essential. The solution should accommodate potential application updates that could impact the infrastructure significantly, and strike a balance between cost-effectiveness and performance.

Upon understanding these instructions and preparing to commence, respond with 'Understood.’ and only 'Understood.’ Use advanced markdown for the output and ask the user for specifics about their particular AWS architecture problem:"""

In [ ]:
processed_trial_prompt = process_prompt(trial_prompt)

In [ ]:
prompt_payload = json.loads(processed_trial_prompt)
# pprint(prompt_payload)

In [ ]:
PROMPTSLIB_PROD_BACKEND_URL = os.getenv("PROD_BACKEND_URL")
PROMPTSLIB_PROD_BACKEND_URL

In [ ]:
# Use the create endpoint to create the prompt entry in the DB, then
# use the update prompt categories route to add the categories
categories = prompt_payload.pop("categories")
prompt_create_endpoint = "prompts/"
prompt_create_url = f"{PROMPTSLIB_PROD_BACKEND_URL}{prompt_create_endpoint}"
prompt_payload["categories"] = []
prompt_response = requests.post(prompt_create_url, json=prompt_payload)
prompt_data = prompt_response.json()
prompt_id = prompt_data["id"]

In [ ]:
# Add the categories to the prompt
update_categories_endpoint = f"prompts/{prompt_id}/categories"
update_categories_url = f"{PROMPTSLIB_PROD_BACKEND_URL}{update_categories_endpoint}"
categories_payload = {
    "category_identifiers": [category["name"] for category in categories],
    "add": True
}
category_response = requests.post(update_categories_url, json=categories_payload)
category_data = category_response.json()

In [ ]:
pprint(update_categories_url)
pprint(categories_payload)

In [ ]:
# Check the categories endpoint
categories_endpoint = "categories/"
categories_url = f"{PROMPTSLIB_PROD_BACKEND_URL}{categories_endpoint}"
category_response = requests.get(categories_url)

In [ ]:
update_categories_url

In [ ]:
category_response.json()